In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

## Simulate Image Sets for Photogrammetry

With these functions, it is possible to generate **image sets to be used with a photogrammetry software**. We suggest to read the [blend-examples notebook](../examples/phg-examples.ipynb) and follow their steps.

[blendjupyter.ipynb](blendjupyter.ipynb) is a shared Jupyter module &#171;written to set up and manipulate cameras (`'bpy.types.Camera'`), scenes (`'bpy.context.scene'`)&#187;, among other things.

This notebook **does not use** a Python kernel, [uses a Blender kernel](https://github.com/panzi/blender_ipython).

In [ ]:
import blendjupyter
import math
import os

In [ ]:
def take_photos(cam, sc, l_loc_rot, **kwargs):
    """Take photos with the cam ('bpy.types.Camera') in the
    coordinates and the angles referenced as tuples of l_loc_rot
    ('list')."""    
    s3path = kwargs.pop('s3path',
                        os.path.join(os.path.expanduser('~'),
                                     'super-scanner-software-s3'))
    s3out = kwargs.pop('s3out',
                       os.path.join(os.path.expanduser('~'),
                                    's3-out'))
    dirname = kwargs.pop('dirname', 'blend-phg-set-0001')
    # Path of the directory that will contain the photos to be used
    # with photogrammetry.
    path = kwargs.pop('path', os.path.join(s3out,
                                           'scanner',
                                           dirname))
    prefix = kwargs.pop('prefix', 'view_')
    ext = kwargs.pop('ext', 'TIFF')
    exif = kwargs.pop('exif',
                      os.path.join(s3path,
                                   'img',
                                   'Photo 25-09-16 11 11 00.jpg'))
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(generate_obj.__name__, list(kwargs.keys())[-1]))
    while True: # Create a new folder for the set in the main path.
        if not os.path.exists(path):
            os.makedirs(path)
            break
        else:
            str_num = path.split('-')[-1]
            num_dir = int(str_num)
            num_dir += 1
            str_num_dir = blendjupyter.num_str_zeros(num_dir,
                                                     len(str_num))
            path = '-'.join(path.split('-')[:-1]) + '-' + str_num_dir
    len_llocrot = len(l_loc_rot)
    for i in range(len_llocrot):
        loc, rot = l_loc_rot[i]
        cam.location = loc
        for j in range(3):  # 3 by each rotation.
            cam.rotation_euler[j] = math.radians(rot[j])
        blendjupyter.shoot_cam(cam,
                               sc,
                               os.path.join(path,
                                            prefix + blendjupyter.num_str_zeros(i,
                                                len(str(len_llocrot)),
                                                matlab=True)),
              exif,
              extension=ext)

In [ ]:
def range_float(a, b, step):
    """Emulate the np.arange() of the numpy Python module.
    
    With this function, IT IS NOT NECESSARY to install numpy for
    Blender."""
    l = list()
    if a > b:
        while a > b:
            l.append(a)
            a -= step
    else:
        if a < b:
            while a < b:
                l.append(a)
                a += step
    return l

In [ ]:
def walk_sphere(**kwargs):
    """Return a list of tuples with the (x, y, z) coordinates and
    rotation angles of a camera that moves along a circular path in
    different heights and radius of a sphere.
    
    Example
    >>> l = walk_sphere()
    >>> l
    [((7.984359711335656, 0.0, 0.5000000000000012),
      (86.41667830152802, 0, 90)),
       ...,
     ((3.3777275704249043, -5.850395766102116, 4.2853034711634965),
      (57.61111886768535, 0, 390.0))]
    >>> (x, y, z), (rotx, roty, rotz) = l[0]
    >>> rotz
    90
    """
    r = kwargs.pop('r', 8)  # The radius in 'NONE' units.
    # Number of photos during movement in xy-plane.
    hsteps = kwargs.pop('hsteps', 16)
    # Number of xy-planes of movement.
    vsteps = kwargs.pop('vsteps', 4)
    # First height of the camera in 'NONE' units.
    hinit = kwargs.pop('hinit', 0.5)
    # Maximum height of the camera. It not should be greater that
    # the radius, r ('float').
    hend = kwargs.pop('hend', 6)
    # In Blender, a simple way to find out the phase change angle is
    # placing the camera ('bpy.types.Camera') in the first (x, y, z)
    # coordinate and manually changing the Y rotation angle of
    # 'XYZ' rotation order.
    dphase = kwargs.pop('dphase', 90)
    daltitude = kwargs.pop('daltitude', 0)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(generate_obj.__name__, list(kwargs.keys())[-1]))
    # Phi is the angle measured from the z-Axis to the xy-plane.
    phi_init = 90 - math.degrees(math.asin(hinit / float(r)))
    phi_end = 90 - math.degrees(math.asin(hend / float(r)))
    phi_step = abs(phi_init - phi_end) / float(vsteps)
    range_phi = range_float(phi_init, phi_end, phi_step)
    range_phi = range_phi[:vsteps]
    theta_step = 360 / float(hsteps)
    range_theta = range_float(0, 360, theta_step)
    range_theta = range_theta[:hsteps]
    l_coord = list()
    for phi in range_phi:
        for theta in range_theta:
            # Parameterization of a sphere [1].
            #
            # [1] Carmen, M. (2007). Parametrizaciones. Retrieved from
            # http://www.dm.uba.ar/materias/complementos_analisis_Mae/2007/2/parametrizaciones.pdf
            x = r * math.cos(math.radians(theta)) * math.sin(math.radians(phi))
            y = r * math.sin(math.radians(theta)) * math.sin(math.radians(phi))
            z = r * math.cos(math.radians(phi))
            rotx = phi + daltitude
            roty = 0
            rotz = theta + dphase
            l_coord.append(((x, y, z), (rotx, roty, rotz)))
    return l_coord